# Tetrahedra in a grid
In this example, we are using clique packing to find the maximal number of vertex disjoint tetrahedra that can
be packed in a grid graph.

In [ ]:
import networkx as nx

from itertools import product, combinations

# GraphILP API: import networkx graphs and use clique packing
from graphilp.imports import networkx as impnx
from graphilp.packing import clique_packing as cp

# Use Matplotlib for plotting our 3d grid graph and the packed tetrahedra
%matplotlib inline
import matplotlib.pyplot as plt

from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

# Allow some interaction in the 3d visualisation
from ipywidgets import interact, IntSlider

## Set up the grid graph

In [ ]:
# choose edge length of grid graph
n = 4

In [ ]:
# start with a standard grid graph
G = nx.grid_graph(dim=(n, n, n))

In [ ]:
# allow different colours for different types of edges
edge_colors = {}
for e in G.edges():
    edge_colors[e] = 'k'

In [ ]:
# extend the grid graph to allow for a nice collection of tetrahedra:
# create new vertices at the cube centres, connect them to the cube vertices, and add diagonals to the cubes
new_edges = []

for node in G.nodes():
    if node[0] < n-1 and node[1] < n-1 and node[2] < n-1:
        # centre points
        new_edges.append(((node[0]+0.5, node[1]+0.5, node[2]+0.5), (node[0], node[1], node[2])))
        new_edges.append(((node[0]+0.5, node[1]+0.5, node[2]+0.5), (node[0], node[1]+1, node[2])))
        new_edges.append(((node[0]+0.5, node[1]+0.5, node[2]+0.5), (node[0], node[1], node[2]+1)))
        new_edges.append(((node[0]+0.5, node[1]+0.5, node[2]+0.5), (node[0], node[1]+1, node[2]+1)))
        new_edges.append(((node[0]+0.5, node[1]+0.5, node[2]+0.5), (node[0]+1, node[1], node[2])))
        new_edges.append(((node[0]+0.5, node[1]+0.5, node[2]+0.5), (node[0]+1, node[1]+1, node[2])))
        new_edges.append(((node[0]+0.5, node[1]+0.5, node[2]+0.5), (node[0]+1, node[1], node[2]+1)))
        new_edges.append(((node[0]+0.5, node[1]+0.5, node[2]+0.5), (node[0]+1, node[1]+1, node[2]+1)))
        
        # cube diagonals
        new_edges.append(((node[0], node[1], node[2]), (node[0]+1, node[1], node[2]+1)))
        new_edges.append(((node[0], node[1], node[2]+1), (node[0]+1, node[1], node[2])))
        new_edges.append(((node[0], node[1]+1, node[2]), (node[0]+1, node[1]+1, node[2]+1)))
        new_edges.append(((node[0], node[1]+1, node[2]+1), (node[0]+1, node[1]+1, node[2])))

        new_edges.append(((node[0], node[1], node[2]), (node[0], node[1]+1, node[2]+1)))
        new_edges.append(((node[0], node[1]+1, node[2]), (node[0], node[1], node[2]+1)))
        new_edges.append(((node[0]+1, node[1], node[2]), (node[0]+1, node[1]+1, node[2]+1)))
        new_edges.append(((node[0]+1, node[1]+1, node[2]), (node[0]+1, node[1], node[2]+1)))

        new_edges.append(((node[0], node[1], node[2]), (node[0]+1, node[1]+1, node[2])))
        new_edges.append(((node[0]+1, node[1], node[2]), (node[0], node[1]+1, node[2])))
        new_edges.append(((node[0], node[1], node[2]+1), (node[0]+1, node[1]+1, node[2]+1)))
        new_edges.append(((node[0]+1, node[1], node[2]+1), (node[0], node[1]+1, node[2]+1)))

G.add_edges_from(new_edges)

In [ ]:
# show additional edges in light grey
for e in G.edges():
    if e not in edge_colors:
        edge_colors[e] = '#AAAAAA'

## Plot the grid graph

In [ ]:
X = [node[0] for node in G.nodes()]
Y = [node[1] for node in G.nodes()]
Z = [node[2] for node in G.nodes()]

In [ ]:
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(X, Y, Z)

for edge in G.edges():
    ax.plot([edge[0][0], edge[1][0]], [edge[0][1], edge[1][1]], [edge[0][2], edge[1][2]], c=edge_colors[edge])

## Set up optimisation problem

In [ ]:
optG = impnx.read(G)

In [ ]:
m = cp.create_model(optG, 4)

## Solve optimisation problem and extract solution

In [ ]:
m.optimize()

In [ ]:
cliques = cp.extract_solution(optG, m)

In [ ]:
# get edges per clique
clique_dict = {}
for edge, clique_no in cliques.items():
    if clique_no > 0:
        if clique_no not in clique_dict:
            clique_dict[clique_no] = []
        clique_dict[clique_no].append(edge)

## Visualise solution

In [ ]:
# each 4-clique can be interpreted as a tetrahedron with four faces
triangles = []
for clique_name, clique in clique_dict.items():
    for triple in combinations(clique, 3):
        triangles.append(triple)

In [ ]:
def update(h = 10.0, w = 390.0):    
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(111, projection='3d')
    ax.view_init(h, w)

    ax.scatter(X, Y, Z)

    for edge in G.edges():
        ax.plot([edge[0][0], edge[1][0]], [edge[0][1], edge[1][1]], [edge[0][2], edge[1][2]], c=edge_colors[edge])

    tri = Poly3DCollection(triangles)
    tri.set_alpha(0.5)
    tri.set_edgecolor('#FF0000')
    ax.add_collection3d(tri)

interact(update, w=IntSlider(min=0, max=360, step=5, value=250), h=IntSlider(min=0, max=90, step=5, value=25));